# 导入日线数据

In [1]:
import os
import json
from datetime import datetime,timedelta
# import mysql.connector
import sqlite3
import shutil

In [4]:
def datetime_from_str(date,time):
    year_s, mon_s, day_s = date.split('/')
    try:
        return datetime(int(year_s), int(mon_s), int(day_s),int(time[:2]),int(time[2:]))
    except:
        print ('日期时间格式解析错误',year_s, mon_s, day_s,time)

#reading RichMinds config file 
config_file = r'C:\RichMinds\Config.json'
f = open(config_file,'r')
config_json =json.load(f)

#Get data loading path
load_path = config_json['daily import path']
#print ('股票一分钟线数据导入程序初始化:')
#print ('请确认导入目录:',load_path )
#while True:
#    input_load_path = input('如需更改路径,请输入新的路径')
#    if len(input_load_path) > 5 and os.path.isdir(os.path.join(input_load_path)):
#        load_path = input_load_path
#        break
#    elif input_load_path == '':
#        break
#    print('错误的文件路径')

#Get log path and create log files
log_path = config_json['log path']
current_datetime =datetime.now()
local_now = '{:%Y-%m-%d_%H%M%S}'.format(current_datetime)
log_file_id = '股票日线导入日志_'+ local_now[:19]
log_file_name = log_file_id + '.txt'
log_file_handle = open(os.path.join(log_path,log_file_name),'wt')

#Get processed path for processed file to be moved, path from config plus log_id
processed_path = config_json['processed path'] + '\\'+ log_file_id
os.mkdir(processed_path)

#get file list from data loading path
filenames = [filename for filename in os.listdir(load_path)
               if os.path.isfile(os.path.join(load_path,filename))]

#open database connection, remove MySQL, change to SQLite
#cnx = mysql.connector.connect(user=config_json['DB server']['user'], 
#                              password=config_json['DB server']['password'],
#                              host=config_json['DB server']['host'],
#                              database=config_json['DB server']['database'])

cnx = sqlite3.connect('richmindsdev.db')

dbcursor = cnx.cursor()



print('---开始导入数据---',file = log_file_handle)
#000001 上证指数 日线 不复权
#      日期	    开盘	    最高	    最低	    收盘	    成交量	    成交额
#2011/10/10,2363.08,2368.15,2338.69,2344.79,41745306,39926411264.00
#2011/10/11,2401.74,2412.38,2323.41,2348.52,63975805,62773514240.00

for filename in filenames:
    print('开始处理文件:',filename,file=log_file_handle)
    filehandler = open(os.path.join(load_path,filename),'rt')
    line_id = 0
    for line in filehandler:
        line_id  = line_id + 1
        line = line.strip()
        if line_id == 1:
            words = line.split(' ')
            stock_id = words[0]
            stock_name = words[1]
#MySQl -> SQLite3
#            add_stock_info = ('''REPLACE INTO stock_basic_info 
#            (Stock_ID, Stock_Name) VALUES (%s, %s)''')
            add_stock_info = '''REPLACE INTO stock_basic_info (Stock_ID, Stock_Name) VALUES (?, ?)'''
            data_stock_info = (stock_id, stock_name)
            # print (add_stock_info,data_stock_info)
            # Insert or replace new stock_info
            dbcursor.execute(add_stock_info,data_stock_info)
        elif line_id ==2: continue
        elif line.startswith('数据来源'): continue
        else:
            words = line.split(',')
            trans_date = words[0]
            trans_time =  '1500'
            trans_datetime = datetime_from_str(trans_date,trans_time)
            open_price = words[1]
            high_price = words[2]
            low_price = words[3]
            close_price = words[4]
            trans_volume = words[5]
            trans_turnover = words[6]
#MySQl -> SQLite3 
#            add_stock_trans_detail = ('''INSERT IGNORE INTO stock_1minute_noauth 
#            (Stock_ID,TransDateTime,Open,High,Low,Close,Volume,Turnover)
#            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)''')
            add_stock_trans_detail = '''INSERT OR IGNORE INTO stock_daily_noauth 
            (Stock_ID,TransDateTime,Open,High,Low,Close,Volume,Turnover)
            VALUES (?,?,?,?,?,?,?,?)'''
            data_stock_trans_detail = (stock_id,trans_datetime,open_price,high_price,low_price,close_price,trans_volume,trans_turnover)
            # Insert or replace new stock_info
            dbcursor.execute( add_stock_trans_detail,data_stock_trans_detail)        
    cnx.commit()
    filehandler.close()
    shutil.move(os.path.join(load_path,filename),os.path.join(processed_path,filename))
        
dbcursor.close()
cnx.close()            
log_file_handle.close()

print ('处理完毕，请检查migration log')


处理完毕，请检查migration log
